In [1]:
import pandas as pd
import re
import nltk
from nltk.tag import pos_tag
from typing import List
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import numpy as np

In [2]:
from gensim.models.word2vec import Word2Vec
import gensim

In [3]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True) 

KeyboardInterrupt: 

In [ ]:
dialogues = pd.read_csv('../code/emo_context_train.txt', sep="\t")
class_counts = dialogues["label"].value_counts()

# Print the class counts
print("class counts: ",class_counts)
dialogues = dialogues[dialogues["label"]!="others"]
labels = dialogues.label
cols = ["turn1", "turn2", "turn3"]
dialogues["combined"] = dialogues[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
dialogues["combined"] = dialogues["combined"].apply(str.lower).apply(word_tokenize).apply(lambda x: " ".join(x))
dialogues["seq_len"] = dialogues["combined"].apply(len)
print(dialogues.columns)

max_len = dialogues["seq_len"].max()
input_text = dialogues.combined.values
tokenized = []
for sen in input_text:
    tokenized.append(sen.split())


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
X = tokenized
y = labels

In [ ]:
train_data, val_data, train_labels, val_labels = train_test_split(tokenized, labels, test_size=0.2, random_state=42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tokenized, y, test_size=0.2, random_state=16)

In [ ]:
train_vectors = []
for sentence in X_train:
    sentence_vec = []
    for word in sentence:
        if word in word2vec_model:
            sentence_vec.append(word2vec_model[word])
    train_vectors.append(np.mean(sentence_vec, axis=0))
    
test_vectors = []
for sentence in X_test:
    sentence_vec = []
    for word in sentence:
        if word in word2vec_model:
            sentence_vec.append(word2vec_model[word])
    test_vectors.append(np.mean(sentence_vec, axis=0))
    
print(len(test_vectors))

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
nb = GaussianNB()
nb.fit(train_vectors, y_train)

In [ ]:
predictions = nb.predict(test_vectors)

In [ ]:
print("accuracy: ",accuracy_score(y_test, predictions))
print("F1 score: ",f1_score(y_test, predictions, average="macro"))

In [ ]:
from sklearn.metrics import classification_report
 
print(classification_report(y_test, predictions))